# Testing the LSTM sentiment-analysis model trained on the IMDB dataset
### Author: Leonardo Espín
#### Date: 4/25/2021

* [Testing on the test data](#Testing-on-the-test-data)
    * [Testing the model through the rest API](#Testing-the-model-through-the-rest-API)
    * [Making a prediction through the rest API](#Making-a-prediction-through-the-rest-API)
* [Testing on arbitrary text](#Testing-on-arbitrary-text)


In [ ]:
import json
import requests
import tensorflow as tf
import sys
sys.path.append("../trainer")
# importing module in trainer folder
import dataprep

In [ ]:
lstm_model = tf.keras.models.load_model('/Users/leoespin/github/imdb-dataset/serve/1/')

In [ ]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160016    
_________________________________________________________________
lstm (LSTM)                  (None, 16)                2112      
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 162,145
Trainable params: 162,145
Non-trainable params: 0
_________________________________________________________________


## Testing on the test data

A value of **0 means a negative review, and 1 means a positive review**. The model returns the probability of the review being positive

In [ ]:
(x_train, y_train), (x_test, y_test) = dataprep.get_and_pad_imdb_dataset(maxlen=250) # default value used during training
print('Input sequence:')
print(x_test[0])
print('corresponding label:')
print(y_test[0])
print('model prediction:')
lstm_model.predict(x_test[None, 0, :]) # None is for adding extra dim to account for the batch size

Input sequence:
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    1  591  202   14   31    6  717   10   10    2    2    5 

array([[0.07337198]], dtype=float32)

* check input dimensions:

In [ ]:
print(x_test[0, :].shape)
print(x_test[None, 0, :].shape)

(250,)
(1, 250)


* Run the `tensorflow/serving` container and load the model (after the service has started succesfully, stop the cell execution to be able to run other cells):

In [ ]:
!docker run -p 8501:8501 \
  --mount type=bind,source=/Users/leoespin/github/imdb-dataset/serve/,target=/models/lstm_model \
  -e MODEL_NAME=lstm_model -t tensorflow/serving

2021-04-26 20:08:15.082095: I tensorflow_serving/model_servers/server.cc:88] Building single TensorFlow model file config:  model_name: lstm_model model_base_path: /models/lstm_model
2021-04-26 20:08:15.082538: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2021-04-26 20:08:15.082626: I tensorflow_serving/model_servers/server_core.cc:587]  (Re-)adding model: lstm_model
2021-04-26 20:08:15.191922: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: lstm_model version: 1}
2021-04-26 20:08:15.191993: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: lstm_model version: 1}
2021-04-26 20:08:15.192020: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: lstm_model version: 1}
2021-04-26 20:08:15.193128: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /models/lstm_model/1
2021-04-26 20:08:15.271145: 

### Testing the model through the rest API

In [ ]:
# Model status API
model_name = 'lstm_model'
x = requests.get(f'http://localhost:8501/v1/models/{model_name}')
print(json.dumps(x.json(), indent=2, sort_keys=True))

{
  "model_version_status": [
    {
      "state": "AVAILABLE",
      "status": {
        "error_code": "OK",
        "error_message": ""
      },
      "version": "1"
    }
  ]
}


In [ ]:
# Model Metadata API
x = requests.get(f'http://localhost:8501/v1/models/{model_name}/metadata')
print(json.dumps(x.json(), indent=2, sort_keys=True))

{
  "metadata": {
    "signature_def": {
      "signature_def": {
        "__saved_model_init_op": {
          "inputs": {},
          "method_name": "",
          "outputs": {
            "__saved_model_init_op": {
              "dtype": "DT_INVALID",
              "name": "NoOp",
              "tensor_shape": {
                "dim": [],
                "unknown_rank": true
              }
            }
          }
        },
        "serving_default": {
          "inputs": {
            "embedding_input": {
              "dtype": "DT_FLOAT",
              "name": "serving_default_embedding_input:0",
              "tensor_shape": {
                "dim": [
                  {
                    "name": "",
                    "size": "-1"
                  },
                  {
                    "name": "",
                    "size": "-1"
                  }
                ],
                "unknown_rank": false
              }
            }
          },
          "method_name

### Making a prediction through the rest API

In [ ]:
headers = {"content-type": "application/json"}
data = json.dumps({"instances": x_test[None, 0, :].tolist()})

In [ ]:
json_response = requests.post('http://localhost:8501/v1/models/lstm_model:predict',
                              data = data, headers = headers)
print(json.dumps(json_response.json(), indent=2, sort_keys=True))

{
  "predictions": [
    [
      0.0733719766
    ]
  ]
}


## Testing on arbitrary text

* preprocess and tokenize sample text:

In [ ]:
# get the word index for the IMDB dataset
imdb_word_index = dataprep.get_imdb_word_index()

good = dataprep.clean_tokenize('I like it, very nice!', imdb_word_index)
bad = dataprep.clean_tokenize('worst movie ever', imdb_word_index)

* get the predictions:

In [ ]:
data = json.dumps({"instances" : [ good[0], bad[0] ]})
json_response = requests.post('http://localhost:8501/v1/models/lstm_model:predict',
                              data = data, headers = headers)
print(json.dumps(json_response.json(), indent=2, sort_keys=True))

{
  "predictions": [
    [
      0.778687477
    ],
    [
      0.0709086359
    ]
  ]
}
